In [442]:
import numpy as np
import os
import pandas as pd
import json
import pickle

from tqdm.notebook import tqdm

In [11]:
dir = '/content/drive/My Drive/Colab Notebooks/Genshin Impact Monte Carlo/'
os.listdir(dir)

['genshin_montecarlo.ipynb', 'banners.json']

In [55]:
# Load banner data from JSON
jsonpath = dir+'banners.json'

with open(jsonpath, 'r') as j:
  all_banners = json.loads(j.read())

In [43]:
print(all_banners)

{'banner': [{'geninfo': {'name': 'Venti', 'prob5': 0.006, 'prob4': 0.051, 'prob3': 0.943, '5startotal': 6, '4startotal': 29}, '5stardrop_char': ['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean'], '4stardrop_char': ['Barbara', 'Fischl', 'Xiangling', 'Sucrose', 'Chongyun', 'Noelle', 'Bennett', 'Ninguang', 'Xingqiu', 'Beidou', 'Razor'], 'rateups': {'5star': [1], '4star': [1, 2, 3]}, 'rateup_perc': {'5star': 0.5, '4star': 0.5}, 'pitypullnum': {'5star': 90, '4star': 10}}]}


In [56]:
# Define the banner class based on JSON data
class Banner():
  def __init__(self, banner):

    self.list_chars = {}

    self.name = banner['geninfo']['name']
    self.prob5 = banner['geninfo']['prob5']
    self.prob4 = banner['geninfo']['prob4']
    self.prob3 = banner['geninfo']['prob3']
    self.total5s = banner['geninfo']['5startotal']
    self.total4s = banner['geninfo']['4startotal']

    self.list_chars['5 star'] = banner['5stardrop_char']
    self.list_chars['4 star'] = banner['4stardrop_char']

    self.rateups = banner['rateups']
    self.rateups_perc = banner['rateup_perc']

    self.pitynum = banner["pitypullnum"]

    self.startup_text()

  def startup_text(self):
    print("Loaded Banner: " + self.name)

    for stars in self.rateups:
      print("Characters on " + stars + " rateup: " + str([self.list_chars[stars][x] for x in self.rateups[stars]]) + " at " + str(self.rateups_perc[stars]*100) + "%")

In [358]:
# Define simulator

class MonteCarloSimulator():
  def __init__(self, Banner, timesrolled=0, nrolls=5, nsims=5):
    self.Banner = Banner
    self.name = self.Banner.name
    self.nrolls = nrolls
    self.nsims = nsims
    self.timesrolled = timesrolled # Number of times already rolled on the banner


    
    self.get4rated = False
    self.get5rated = False

    self.simresults = []
    self.simlist = []

  def _roll1(self)->str:
    roll = ""
    thresh5 = self.Banner.prob5*1000
    thresh4 = self.Banner.prob4*1000
    whichchar = np.random.randint(0, 1000)
    if whichchar <= thresh5:
      roll = self._rollforstar('5 star')
    elif whichchar <= thresh4:
      roll = self._rollforstar('4 star')
    else:
      roll = '3 star'
    return roll

    

  def _rollforstar(self, star)->str:
    # roll = -1

    if self.get5rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['5 star']))
      self.get5rated = False
    elif self.get4rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['4 star']))
      self.get4rated = False

    israted = np.random.randint(0, 1000)
    # print(israted)
    if israted < self.Banner.rateups_perc[star]*1000:
      # print('Get rated')
      # Get a rated up char!
      whichchar = np.random.randint(0, len(self.Banner.rateups[star]))
      # rolledchar = self.Banner.total5s[whichchar]
    else:
      # Get an unrated-up char!
      # print('Get unrated')
      whichchar = np.random.randint(0, len(self.Banner.list_chars[star]))
      # print(whichchar)
      while whichchar in self.Banner.rateups[star]:
        whichchar = np.random.randint(0, len(self.Banner.list_chars[star]))
      
    return self.Banner.list_chars[star][whichchar]
      



  def _sim1(self)->(dict, list):

    results = {}
    simorder = []

    simrolled = self.timesrolled

    pity5cnt = self.timesrolled%self.Banner.pitynum['5 star']
    pity4cnt = self.timesrolled%self.Banner.pitynum['4 star']


    if simrolled>0 and ((simrolled+1) % self.Banner.pitynum['5 star'] == 0):
      guarantee5 = True
    if simrolled>0 and ((simrolled+1) % self.Banner.pitynum['4 star'] == 0):
      guarantee4 = True
    else:
      guarantee5, guarantee4 = False, False
    
    for i in range(self.nrolls):
      # print("New Roll! " + str(guarantee5) + " | " + str(guarantee4))
      roll = ""
      if guarantee5:
        # print('i: ' + str(i) + ' 5star pity')
        roll = self._rollforstar('5 star')
        pity5cnt = 0
        guarantee5 = False
      elif guarantee4:
        roll = self._rollforstar('4 star')
        pity4cnt = 0
        guarantee4 = False
      else:        
        roll = self._roll1()
        if roll in self.Banner.list_chars['5 star']:
          # print('Natural 5 stopping pity at: !' + str(pity5cnt))
          pity5cnt = 0
        elif roll in self.Banner.list_chars['4 star']:
          # print('Natural 4 stopping pity at: !' + str(pity4cnt))
          pity4cnt = 0


      # print(roll)

      simorder.append(roll)

      if roll not in results.keys():
        results[roll] = 1
        if roll in self.Banner.list_chars['5 star'] and roll not in self.Banner.rateups['5 star'] and not self.get5rated:
          self.get5rated = True
        elif roll in self.Banner.list_chars['4 star'] and roll not in self.Banner.rateups['4 star'] and not self.get4rated:
          self.get4rated = True
      else:
        results[roll] = results[roll] + 1
      
      simrolled += 1
      pity5cnt += 1
      pity4cnt += 1

      if ((pity5cnt+1) % self.Banner.pitynum['5 star']) == 0:
        # print('pity time 5 ')
        guarantee5 = True
      if ((pity4cnt+1) % self.Banner.pitynum['4 star']) == 0:
        guarantee4 = True

      # print(self.Banner.pitynum['5 star'])
      # print(pity5cnt, pity5cnt+1%self.Banner.pitynum['5 star'])

    return results, simorder

  def runallsims(self):
    for i in tqdm(range(self.nsims)):
      single_sim, single_simorder = self._sim1()
      self.simresults.append(single_sim)
      self.simlist.append(single_simorder)

  def testroll(self):
    res = {}
    for i in range(10):
      roll = self._rollforstar('4 star')
      if roll not in res.keys():
        res[roll] = 1
      else:
        res[roll] = res[roll] + 1
    print(res)


In [ ]:
# Define a single roll

In [446]:
# Testing
b = Banner(all_banners['banner'][0])
sim = MonteCarloSimulator(b, nrolls=1000, nsims=200000)

Loaded Banner: Venti
Characters on 5 star rateup: ['Venti'] at 50.0%
Characters on 4 star rateup: ['Barbara', 'Fischl', 'Xiangling'] at 50.0%


In [447]:
print(sim.Banner.list_chars['5 star'])
sim.testroll()

['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean']
{'Chongyun': 1, 'Xiangling': 1, 'Xingqiu': 2, 'Noelle': 3, 'Fischl': 1, 'Barbara': 1, 'Beidou': 1}


In [448]:
sim.runallsims()

In [449]:
# Turn sims into a pandas dataframe

simdf = pd.DataFrame(sim.simresults)
simdf = simdf.fillna(0)
simdf['5 star'] = simdf[sim.Banner.list_chars['5 star']].sum(axis=1)
simdf['4 star'] = simdf[sim.Banner.list_chars['4 star']].sum(axis=1)
simdf['Total Pulls'] = simdf[simdf.columns[:-2]].sum(axis=1)
simdf

,3 star,Barbara,Xiangling,Diluc,Fischl,Sucrose,Noelle,Qiqi,Ninguang,Bennett,Venti,Razor,Chongyun,Xingqiu,Mona,Beidou,Keqing,Jean,5 star,4 star,Total Pulls
0,853,21,24,2.0,24,10.0,4.0,2.0,4.0,11.0,10.0,9.0,10.0,5.0,2.0,8.0,1.0,0.0,17.0,130.0,1000.0
1,853,23,20,0.0,22,6.0,6.0,1.0,10.0,11.0,8.0,16.0,5.0,12.0,0.0,5.0,2.0,0.0,11.0,136.0,1000.0
2,858,22,24,1.0,20,8.0,6.0,0.0,11.0,9.0,7.0,9.0,8.0,7.0,2.0,6.0,2.0,0.0,12.0,130.0,1000.0
3,858,25,18,0.0,21,8.0,9.0,1.0,10.0,6.0,8.0,9.0,4.0,8.0,0.0,10.0,3.0,2.0,14.0,128.0,1000.0
4,855,22,15,0.0,26,7.0,10.0,2.0,8.0,3.0,9.0,9.0,9.0,16.0,0.0,6.0,0.0,3.0,14.0,131.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,854,22,23,2.0,28,7.0,5.0,1.0,7.0,9.0,4.0,6.0,9.0,7.0,1.0,10.0,3.0,2.0,13.0,133.0,1000.0
199996,858,25,19,1.0,15,10.0,9.0,1.0,7.0,10.0,6.0,9.0,6.0,11.0,5.0,5.0,1.0,2.0,16.0,126.0,1000.0
199997,848,25,17,1.0,30,10.0,7.0,2.0,8.0,5.0,9.0,6.0,8.0,8.0,2.0,9.0,4.0,1.0,19.0,133.0,1000.0
199998,861,25,17,2.0,22,7.0,10.0,0.0,8.0,8.0,5.0,9.0,6.0,5.0,0.0,10.0,3.0,2.0,12.0,127.0,1000.0


In [450]:
# Determine the condition
simslice = simdf[simdf['5 star'] > 6]
simslice

,3 star,Barbara,Xiangling,Diluc,Fischl,Sucrose,Noelle,Qiqi,Ninguang,Bennett,Venti,Razor,Chongyun,Xingqiu,Mona,Beidou,Keqing,Jean,5 star,4 star,Total Pulls
0,853,21,24,2.0,24,10.0,4.0,2.0,4.0,11.0,10.0,9.0,10.0,5.0,2.0,8.0,1.0,0.0,17.0,130.0,1000.0
1,853,23,20,0.0,22,6.0,6.0,1.0,10.0,11.0,8.0,16.0,5.0,12.0,0.0,5.0,2.0,0.0,11.0,136.0,1000.0
2,858,22,24,1.0,20,8.0,6.0,0.0,11.0,9.0,7.0,9.0,8.0,7.0,2.0,6.0,2.0,0.0,12.0,130.0,1000.0
3,858,25,18,0.0,21,8.0,9.0,1.0,10.0,6.0,8.0,9.0,4.0,8.0,0.0,10.0,3.0,2.0,14.0,128.0,1000.0
4,855,22,15,0.0,26,7.0,10.0,2.0,8.0,3.0,9.0,9.0,9.0,16.0,0.0,6.0,0.0,3.0,14.0,131.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199995,854,22,23,2.0,28,7.0,5.0,1.0,7.0,9.0,4.0,6.0,9.0,7.0,1.0,10.0,3.0,2.0,13.0,133.0,1000.0
199996,858,25,19,1.0,15,10.0,9.0,1.0,7.0,10.0,6.0,9.0,6.0,11.0,5.0,5.0,1.0,2.0,16.0,126.0,1000.0
199997,848,25,17,1.0,30,10.0,7.0,2.0,8.0,5.0,9.0,6.0,8.0,8.0,2.0,9.0,4.0,1.0,19.0,133.0,1000.0
199998,861,25,17,2.0,22,7.0,10.0,0.0,8.0,8.0,5.0,9.0,6.0,5.0,0.0,10.0,3.0,2.0,12.0,127.0,1000.0


In [451]:
# Analyze and get data

print(simdf.mean())
print((len(simslice.index)/sim.nsims)*100)



3 star          853.827290
Barbara          21.974130
Xiangling        21.975290
Diluc             1.423990
Fischl           22.005615
Sucrose           8.240775
Noelle            8.235930
Qiqi              1.431005
Ninguang          8.253100
Bennett           8.243435
Venti             7.130925
Razor             8.241090
Chongyun          8.248920
Xingqiu           8.239275
Mona              1.429150
Beidou            8.248860
Keqing            1.425970
Jean              1.425250
5 star           14.266290
4 star          131.906420
Total Pulls    1000.000000
dtype: float64
100.0


In [452]:
filename_extension = '_s' + str(int(sim.nsims/1000)) + 'k_r' + str(sim.nrolls)

with open(dir+'simres'+filename_extension + '.pkl', 'wb') as f:
  pickle.dump(sim.simresults, f)

with open(dir+'simorder'+filename_extension+'.pkl', 'wb') as f:
  pickle.dump(sim.simlist, f)



In [453]:
os.listdir(dir)

['genshin_montecarlo.ipynb',
 'banners.json',
 'simres_s10k_r210.pkl',
 'simorder_s10k_r210.pkl',
 'simres_s200k_r1000.pkl',
 'simorder_s200k_r1000.pkl']